<a href="https://colab.research.google.com/github/mikaelvesavuori/tensorflow-object-detection-retraining-coral-usb-support/blob/master/Object_detection_and_retraining_with_support_for_Coral_USB_Accelerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object detection and retraining with support for Coral USB Accelerator
# Introduction

This notebook helps you retrain an existing COCO (Common Objects in Context) model, and output it with Tensorflow Lite to be a quantized model that can do object detection on your custom labels. Quantized models will be compatible with an Edge TPU device like the Coral USB Accelerator, given that the original model is supported by one.

This entire notebook is to a large extent built on fantastic previous work that has been done by contributors globally. The section on preparation is based on https://blog.ml6.eu/our-edge-tpu-demo-project-cbc9bea5a355, and the training section is adapted from https://github.com/dctian/DeepPiCar/blob/master/models/object_detection/code/tensorflow_traffic_sign_detection.ipynb.

At the time of writing (late April 2020), coming in as a newbie in hardware-accelerated machine learning can certainly be a confusing experience. Some of the factors delaying taking those baby steps are:

- Tensorflow version 2 is out since some time back, but seems unequivocally rejected for actual object detection use. Not sure why that is.
- Overall learning curve is high, further complicated by a mix of Python (which I've only encountered in bits before), tons of different writing styles and libraries, as well as the TF v1 and TF v2 conflict mentioned above.
- Using the Coral USB Accelerator, you will find a number of examples online (including the very good documentation) but for me who learns best from articles and examples there are not very many "working" examples that I myself can explain. Some are hidden behind large scripts and until quite some time later, I never understood that there were "built-in" scripts that come with the TF models.
- And of course: All of the specificities around what can and cannot compile to an Edge TPU, including the range of opaque and mystical errors that the ML world can produce when trying to find one's way.
- Finally, I wanted an example in notebook format, ideally Google Colab, since it's a more stable ML platform than I am managing to set up my Macbook Pro to be.

I hope that this notebook will serve as a complete, well-documented example for how to prepare data and then train it, and finally to compile it for your Coral USB Accelerator. You should be able to find good information here, even if you are not interested in all of the steps.

My intent is not to take any credit (since again, most material here is simply repurposed and adjuster), beyond hopefully whatever gratefulness that you may feel if I manage to make learning this stuff easier and faster.

With that said, I might not have covered all the most minute details. Expect to have to fill in some blanks as you go.

PS: Obviously, as time passes and TF v2 settles as the standard, we will have to see whether this notebook remains relevant in 2021 and beyond.

**Requirements for compiling to Edge TPU devices**

The two basic requirements are:

1) To train a model, you **should ideally** use a pre-trained model (suggestion: use the [Tensorflow model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md))

2) The model **must** be quantized

**Note on using Coral USB Accelerator with Raspberry Pi Zero W**

The above list should give you a sense that the ML landscape is shifting quite fast, and because we are dealing with both software and hardware, things change on a frequent basis.

One of my failed attempts while working on this was getting the Coral stick to work with Raspberry Pi Zero W. It's technically possible, but you would need to compile for the version 10 runtime. This is because the Zero uses an ARMv6 architecture, while TF (if I remember correctly) is compiled for ARMv7. Yes, you can rebuild/recompile, but that's out of my league. Ultimately I put that side-project on ice. So:

- Yes, Coral works on the Pi Zero W given the official demos
- No, it's not very easy (for me at least) to compile custom models and get them working on the Zero W

**References**

- https://blog.ml6.eu/our-edge-tpu-demo-project-cbc9bea5a355
- https://gist.github.com/StanCallewaert
- https://github.com/dctian/DeepPiCar/blob/master/models/object_detection/code/tensorflow_traffic_sign_detection.ipynb
- https://github.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi
- https://github.com/toborobot/train_own_tensorflow_colab/blob/master/train_detect_own_dataset_tf_colab.ipynb
- https://www.dlology.com/blog/how-to-train-an-object-detection-model-easy-for-free/
- https://github.com/datitran/raccoon_dataset
- https://pythonprogramming.net/creating-tfrecord-files-tensorflow-object-detection-api-tutorial/

**Resources**

- [Retrain an object detection model](https://coral.ai/docs/edgetpu/retrain-detection/)
- [Train object detection model on Google Colab with TPU](https://stackoverflow.com/questions/59210930/train-object-detection-model-on-google-colab-with-tpu)

**Tensorflow: GitHub Models Repository Commits**

We are going to use Tensorflow 1.15, so it's important that models and the library itself is not "latest".

From https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10:

"If you are using an older version of TensorFlow, here is a table showing which GitHub commit of the repository you should use. I generated this by going to the release branches for the models repository and getting the commit before the last commit for the branch. (They remove the research folder as the last commit before they create the official version release.)"

- TF v1.7:	https://github.com/tensorflow/models/tree/adfd5a3aca41638aa9fb297c5095f33d64446d8f
- TF v1.8:	https://github.com/tensorflow/models/tree/abd504235f3c2eed891571d62f0a424e54a2dabc
- TF v1.9:	https://github.com/tensorflow/models/tree/d530ac540b0103caa194b4824af353f1b073553b
- TF v1.10:	https://github.com/tensorflow/models/tree/b07b494e3514553633b132178b4c448f994d59df
- TF v1.11:	https://github.com/tensorflow/models/tree/23b5b4227dfa1b23d7c21f0dfaf0951b16671f43
- TF v1.12:	https://github.com/tensorflow/models/tree/r1.12.0
- TF v1.13:	https://github.com/tensorflow/models/tree/r1.13.0
- Latest version:	https://github.com/tensorflow/models

# 1. Prepare data on local machine

## Assumptions

- Films recorded with iPhone
- Mac or Linux computer running bash/zsh
- You have [Python 3](https://www.python.org/downloads/) installed
- You have [Homebrew](https://brew.sh/) pre-installed (else go ahead and install it)

Steps should be able to be replicated on Windows or other operating systems, but you will have to figure that out for yourself.

## File and folder structure of data

The below tree shows how your data should be structured:

```
[NAME]
  - images
    - test
    - train
```

Annotations should be in Pascal VOC format, saved as XML files.

We will assume the use of `ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03` as the basis for our model, so let's go with images scaled to 300x300 pixels as well.

## Create material

For the next step, you should film a number of objects. I will assume you are using videos shot with an iPhone. Obviously, adapt file formats and whatever you need for your use case.

### Optional: Download sample data

If you just want to try this out, or verify that the steps work, or if you are just along for the ride — then go ahead and download some sample data. If doing so, you can skip the coming parts and go straight to "Create model (inside Google Colab)".

In [0]:
%cd /content
!mkdir -p your_material
!curl -O -L https://material/your_material.zip
!unzip your_material.zip
!cd /content/your_material

## Prepare data on your local machine

We are going to split the videos into images, then resize/compress the images. After that, you'll begin labeling the details you are after.

Install `ffmpeg` to process your videos/images.

In [0]:
brew install ffmpeg

/bin/bash: brew: command not found


For each of the videos, convert them to MP4 format.

In [0]:
for name in *.mov; do
  ffmpeg -nostdin -i "$name" -vcodec h264 -acodec mp2 "$name.mp4"
done

Start splitting videos into images.

In [0]:
FILENAME=("somefile1" "somefile2" "somefile3")
FPS=3

for i in "${FILENAME[@]}"
do
  VIDEO_LENGTH_IN_SECONDS=ffprobe -i "$i.mp4" -show_entries format=duration -v quiet -of csv="p=0"
  ffmpeg -i "$i.mp4" -vf scale=300:-1,crop=300:300,fps=$FPS $i%04d.jpg
done

Remove any poor and/or incomplete images.

### Install labeling software and label your images

Place the image files in a `test` folder.

Now: Go ahead and label all images.

Short keys are `W` for selecting the box tool, `D` for next image, and `A` for the previous image.

I suggest turning on `Single Class Mode` (if applicable to your use case) and setting a default label to minimize clicks.

In [0]:
git clone git@github.com:tzutalin/labelImg.git
cd labelImg
brew install qt
brew install libxml2
make qt5py3
python3 labelImg.py

Cloning into 'labelImg'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
bash: line 2: cd: labelImg: No such file or directory
bash: line 3: brew: command not found
bash: line 4: brew: command not found
make: *** No rule to make target 'qt5py3'.  Stop.
python3: can't open file 'labelImg.py': [Errno 2] No such file or directory


Then proceed to shuffling out (or just randomly picking) about 10-20% of your images and place them in a `test` folder.

Zip `test` and `train` into an archive and upload it into the Colab file browser.

# 2. Create model (inside Google Colab)

At this point, we can actually start doing the heavy lifting. All of the following steps should be done in the cloud. I will assume that you are reading this in a Colab notebook, meaning you can start running the cells.

## Setup system

### Install required packages

In [0]:
# Force TF v1 since this is the one that actually works for object detection (as of April 2020)
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.2


In [0]:
# Install dependencies and TF models, and set up the Python path
# Do a test to verify it all functions correctly
%cd /content
! git clone --quiet https://github.com/tensorflow/models.git

! apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

! pip install -q Cython contextlib2 pillow lxml matplotlib

! pip install -q pycocotools

%cd /content/models/research
! protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

! python object_detection/builders/model_builder_test.py

/content
fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_confi

### Configuration

In [0]:
# We will use code from David Tian and his repo "DeepPiCar" for some of the bulk processing
repo_url = 'https://github.com/dctian/DeepPiCar'
model_dir = '/content/training'

# Number of training steps
num_steps = 500 # You will want to go much higher when you've verified that settings work

# Number of evaluation steps
num_eval_steps = 50 # Eval steps should probably be 10-20% of your regular steps count

# Model configs from the Tensorflow model zoo GitHub repo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models
MODELS_CONFIG = {
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18.tar.gz
    'ssd_mobilenet_v1_quantized': {
        'model_name': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18',
        'pipeline_file': 'ssd_mobilenet_v1_quantized_300x300_coco14_sync.config',
        'batch_size': 12
    },
    #http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
    'ssd_mobilenet_v2_quantized': {
        'model_name': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03',
        'pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'batch_size': 12
    },
}

# Pick the model you want to use - select a model from above; add more if you like!
selected_model = 'ssd_mobilenet_v2_quantized'

# Name of the object detection model to use
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipeline file in Tensorflow's object detection API
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size should fits in Colab's GPU memory
batch_size = MODELS_CONFIG[selected_model]['batch_size']

In [0]:
import os

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

# Clone the DeepPiCar repo
! git clone {repo_url}
%cd {repo_dir_path}

! git pull

/content
Cloning into 'DeepPiCar'...
remote: Enumerating objects: 1027, done.
remote: Total 1027 (delta 0), reused 0 (delta 0), pack-reused 1027
Receiving objects: 100% (1027/1027), 141.14 MiB | 11.07 MiB/s, done.
Resolving deltas: 100% (320/320), done.
/content/DeepPiCar
Already up to date.


## Setup for training

### Create CSV from XML and TFRecords

Unzip your images.

Note that `images.zip` is hardcoded below – change that filename if your filename differs.

In [0]:
%cd {repo_dir_path}/models/object_detection

! rm -rf data/annotations
! rm -rf data/images

! mkdir -p data/images

%cd /content

! rm -rf sample_data

! unzip images.zip -d $repo_dir_path/models/object_detection/data/images

/content/DeepPiCar/models/object_detection
/content
Archive:  images.zip
   creating: /content/DeepPiCar/models/object_detection/data/images/test/
  inflating: /content/DeepPiCar/models/object_detection/data/images/test/IMG_0245.jpeg  
  inflating: /content/DeepPiCar/models/object_detection/data/images/__MACOSX/test/._IMG_0245.jpeg  
  inflating: /content/DeepPiCar/models/object_detection/data/images/test/IMG_0241.xml  
  inflating: /content/DeepPiCar/models/object_detection/data/images/test/IMG_0245.xml  
  inflating: /content/DeepPiCar/models/object_detection/data/images/test/IMG_0237.xml  
  inflating: /content/DeepPiCar/models/object_detection/data/images/test/IMG_0241.jpeg  
  inflating: /content/DeepPiCar/models/object_detection/data/images/__MACOSX/test/._IMG_0241.jpeg  
  inflating: /content/DeepPiCar/models/object_detection/data/images/test/IMG_0237.jpeg  
  inflating: /content/DeepPiCar/models/object_detection/data/images/__MACOSX/test/._IMG_0237.jpeg  
   creating: /content/

Create records and files.

In [0]:
%cd {repo_dir_path}/models/object_detection

# Convert train folder annotation XML files to a single CSV file, and generate the `label_map.pbtxt` file to `data/` directory as well
! python code/xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation XML files to a single CSV
! python code/xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
! python code/generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
! python code/generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

/content/DeepPiCar/models/object_detection
Successfully converted xml to csv.
Generate `data/annotations/label_map.pbtxt`
Successfully converted xml to csv.


W0504 19:27:08.131909 140089161697152 module_wrapper.py:139] From code/generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0504 19:27:08.140185 140089161697152 module_wrapper.py:139] From /content/models/research/object_detection/utils/label_map_util.py:138: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/DeepPiCar/models/object_detection/data/annotations/train.record


W0504 19:27:12.943895 140707979683712 module_wrapper.py:139] From code/generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0504 19:27:12.950845 140707979683712 module_wrapper.py:139] From /content/models/research/object_detection/utils/label_map_util.py

In [0]:
# Set names and paths
test_record_fname = repo_dir_path + '/models/object_detection/data/annotations/test.record'
train_record_fname = repo_dir_path + '/models/object_detection/data/annotations/train.record'
label_map_pbtxt_fname = repo_dir_path + '/models/object_detection/data/annotations/label_map.pbtxt'

In [0]:
# Verify labels
!cat data/annotations/test_labels.csv

filename,width,height,class,xmin,ymin,xmax,ymax
IMG_0237.jpeg,300,300,drawotype-image,107,19,205,94
IMG_0237.jpeg,300,300,drawotype-heading,107,123,184,157
IMG_0237.jpeg,300,300,drawotype-paragraph,102,174,201,218
IMG_0237.jpeg,300,300,drawotype-button,102,241,142,273
IMG_0237.jpeg,300,300,drawotype-button,153,241,196,276
IMG_0245.jpeg,300,300,drawotype-image,98,29,233,103
IMG_0245.jpeg,300,300,drawotype-paragraph,107,126,212,176
IMG_0245.jpeg,300,300,drawotype-heading,103,195,214,251
IMG_0241.jpeg,300,300,drawotype-paragraph,52,132,245,211


### Download pretrained model

In [0]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile

MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [0]:
# Verify contents
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/models/research/pretrained_model
total 204M
drwx------  2 303230 5000 4.0K Jan  4  2019 .
drwxr-xr-x 63 root   root 4.0K May  4 19:27 ..
-rw-------  1 303230 5000  93M Jan  4  2019 model.ckpt.data-00000-of-00001
-rw-------  1 303230 5000  68K Jan  4  2019 model.ckpt.index
-rw-------  1 303230 5000  20M Jan  4  2019 model.ckpt.meta
-rw-------  1 303230 5000 4.3K Jan  4  2019 pipeline.config
-rw-------  1 303230 5000  24M Jan  4  2019 tflite_graph.pb
-rw-------  1 303230 5000  68M Jan  4  2019 tflite_graph.pbtxt


In [0]:
# Set "fine tune" checkpoint
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/models/research/pretrained_model/model.ckpt'

### Configure training pipeline

In [0]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [0]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [0]:
import re

# Training pipeline file defines:
# - pretrain model path
# - the train/test sets
# - ID to Label mapping and number of classes
# - training batch size
# - epochs to trains
# - learning rate
# - etc

# Below we will use a sample file and make changes to that one

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint: downloaded pre-trained model checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test, which we created earlier with our training/test sets
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path: ID to label file
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps (Number of epochs to train)
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [0]:
! cat {label_map_pbtxt_fname}

item {
    id: 1
    name: 'drawotype-button'
}

item {
    id: 2
    name: 'drawotype-heading'
}

item {
    id: 3
    name: 'drawotype-image'
}

item {
    id: 4
    name: 'drawotype-paragraph'
}

In [0]:
! cat {pipeline_fname}

# Quantized trained SSD with Mobilenet v2 on MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 4
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspe

## Train model

Set `num_train_steps` and `num_eval_steps` values to specify how far you want to take the training.



In [0]:
# Downgrade to fix issues that may happen with conversions between float64 and int
! sudo pip install -U numpy==1.16.0.

     |████████████████████████████████| 17.3MB 1.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.3
    Uninstalling numpy-1.18.3:
      Successfully uninstalled numpy-1.18.3


In [0]:
! python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir='{model_dir}' \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0504 19:28:31.106426 139712329959296 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0504 19:28:31.110123 139712329959296 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0504 19:28:31.110285 139712329959296 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0504 19:28:31.110418 139712329959296 module_wrapper.py:139] From /content/models/resea

Download the "saved_model.pb" variant for possible usage later.

### Optional: Tensorboard visualisation

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip

In [0]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir "{}" --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Export trained model

In [0]:
import os
import re
import numpy as np

output_directory = '%s/fine_tuned_model' % model_dir
os.makedirs(output_directory, exist_ok=True)

In [0]:
lst = os.listdir(model_dir)
# find the last model checkpoint file, i.e. model.ckpt-1000.meta
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

/content/training/model.ckpt-500


In [0]:
! echo creates the frozen inference graph in fine_tune_model
# there is an "Incomplete shape" message.  but we can safely ignore that.
# input_type was originally "image_tensor"; can also "encoded_image_string_tensor"
! python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory='{output_directory}' \
    --trained_checkpoint_prefix='{last_model_path}'

creates the frozen inference graph in fine_tune_model
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0504 19:43:13.462886 140502830004096 module_wrapper.py:139] From /content/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0504 19:43:13.469287 140502830004096 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0504 19:43:13.469519 140502830004096 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:121: The name tf.placeholder is 

In [0]:
# https://medium.com/tensorflow/training-and-serving-a-realtime-mobile-object-detector-in-30-minutes-with-cloud-tpus-b78971cf1193
# create the tensorflow lite graph
! python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path={pipeline_fname} \
    --trained_checkpoint_prefix='{last_model_path}' \
    --output_directory='{output_directory}' \
    --add_postprocessing_op=true

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0504 19:39:41.066137 139985851103104 module_wrapper.py:139] From /content/models/research/object_detection/export_tflite_ssd_graph.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0504 19:39:41.069995 139985851103104 module_wrapper.py:139] From /content/models/research/object_detection/export_tflite_ssd_graph_lib.py:193: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0504 19:39:41.070258 139985851103104 module_wrapper.py:139] From /content/models/research/object_detection/export_tflite_ssd_graph_lib.py:237: The name tf.placeholder is deprecated. Ple

In [0]:
! echo "CONVERTING frozen graph to quantized TF Lite file..."
! tflite_convert \
  --output_file='{output_directory}/model_quantized.tflite' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

CONVERTING frozen graph to quantized TF Lite file...
2020-05-04 19:39:58.702584: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-04 19:39:58.724733: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-04 19:39:58.725351: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P4 major: 6 minor: 1 memoryClockRate(GHz): 1.1135
pciBusID: 0000:00:04.0
2020-05-04 19:39:58.725627: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-04 19:39:58.729214: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-05-04 19:39:58.730750: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] S

In [0]:
! echo "CONVERTING frozen graph to unquantized TF Lite file..."
! tflite_convert \
  --output_file='{output_directory}/model_float.tflite' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops 

CONVERTING frozen graph to unquantized TF Lite file...
2020-05-04 19:40:12.730567: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-05-04 19:40:12.752357: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-05-04 19:40:12.753575: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla P4 major: 6 minor: 1 memoryClockRate(GHz): 1.1135
pciBusID: 0000:00:04.0
2020-05-04 19:40:12.753912: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-05-04 19:40:12.755585: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-05-04 19:40:12.757167: I tensorflow/stream_executor/platform/default/dso_loader.cc:44]

In [0]:
print(output_directory)
! ls -ltra '{output_directory}'
#pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb") # this is main one
#pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")  # this is tflite graph
#pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph_encoded_image_string_tensor.pb")  # this is tflite graph
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph_image_tensor.pb")  # this is tflite graph
! cp '{label_map_pbtxt_fname}' '{output_directory}'

/content/training/fine_tuned_model
total 153200
drwxr-xr-x 5 root root     4096 May  4 19:38 ..
-rw-r--r-- 1 root root 19672075 May  4 19:38 frozen_inference_graph_encoded_image_string_tensor.pb
drwxr-xr-x 3 root root     4096 May  4 19:39 saved_model
-rw-r--r-- 1 root root     4294 May  4 19:39 pipeline.config
-rw-r--r-- 1 root root 19277304 May  4 19:39 tflite_graph.pb
-rw-r--r-- 1 root root 53803577 May  4 19:39 tflite_graph.pbtxt
-rw-r--r-- 1 root root  4760576 May  4 19:40 model_quantized.tflite
-rw-r--r-- 1 root root 18661692 May  4 19:40 model_float.tflite
drwxr-xr-x 2 root root     4096 May  4 19:42 .ipynb_checkpoints
-rw-r--r-- 1 root root 18791804 May  4 19:43 model.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root    23537 May  4 19:43 model.ckpt.index
-rw-r--r-- 1 root root       77 May  4 19:43 checkpoint
-rw-r--r-- 1 root root  2181901 May  4 19:43 model.ckpt.meta
-rw-r--r-- 1 root root 19652631 May  4 19:43 frozen_inference_graph_image_tensor.pb
drwxr-xr-x 4 root root     

## Test the model

### Run inference

The below will use your `repo_dir_path + "models/object_detection/data/images/test"` folder. Make sure that you don't have more than maybe 10 images there, or it will take forever to go through them.

In [0]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname
print(PATH_TO_CKPT)

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR = "/content/"
# PATH_TO_TEST_IMAGES_DIR = os.path.join(repo_dir_path, "models/object_detection/data/images/test")

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.jpeg"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(
    PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

/content/training/fine_tuned_model/frozen_inference_graph_image_tensor.pb
['/content/IMG_0251.jpeg']


In [0]:
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util
from object_detection.utils import ops as utils_ops
from PIL import Image
from matplotlib import pyplot as plt
from io import StringIO
from collections import defaultdict
import zipfile
import tensorflow as tf
import tarfile
import sys
import six.moves.urllib as urllib
import os
import numpy as np
%cd /content/models/research/object_detection


# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")


# This is needed to display the images.
%matplotlib inline


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

ModuleNotFoundError: ignored

In [0]:
# Old, original version

%matplotlib inline

print('Running inferences on %s' % TEST_IMAGE_PATHS)
for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=2)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

Running inferences on ['/content/IMG_0251.jpeg']


TypeError: ignored

In [0]:
# Updated version, based on: https://stackoverflow.com/questions/45283010/tensorflow-object-detection-api-print-detected-class-as-output-to-terminal

with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')


      # Actual detection
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})


      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks'),
          use_normalized_coordinates=True,
          line_thickness=8)
      #plt.figure(figsize=IMAGE_SIZE)
      #plt.imshow(image_np)


      boxes = output_dict['detection_boxes']
      max_boxes_to_draw = boxes.shape[0]
      scores = output_dict['detection_scores']
      min_score_thresh=.7

      for i in range(min(max_boxes_to_draw, boxes.shape[0])):
          if scores is None or scores[i] > min_score_thresh:
              _class_int = int(classes[0][i])
              _class_category = category_index.get(_class_int)
              _key, _value = _class_category.items()
              category = ''.join(_value)[4:]
              print ("Class:", category)
              print ("Score:", scores[i])
              print ("Box:", boxes[i])

Class: drawotype-image
Score: 0.991632
Box: [0.05237004 0.34054607 0.29484951 0.7358442 ]
Class: drawotype-paragraph
Score: 0.9886473
Box: [0.66346896 0.32420185 0.86520004 0.7680491 ]
Class: drawotype-heading
Score: 0.7461554
Box: [0.38533986 0.33156186 0.58825797 0.7326639 ]


In [0]:
import os
import re
import numpy as np

output_directory = '%s/fine_tuned_model' % model_dir
os.makedirs(output_directory, exist_ok=True)

# 3. Adapt the model for Coral Edge TPU

You will now use the [Edge TPU compiler](https://coral.withgoogle.com/web-compiler/) to convert the quantized tflite file into a version that Coral can use. There are definitely good places to read about the technical details of this, but the above training _should_ have successfully resulted in something that you can compile.

In [0]:
# Install the compiler

! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt update

! sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  18138      0 --:--:-- --:--:-- --:--:-- 18138
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [91.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,332 B]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:6 https://packages.cloud.google.com/apt coral-edgetpu-stable/main amd64 Packages [1,277 B]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:9 https://developer.downloa

Let's compile it!

In [0]:
%cd /content/training/fine_tuned_model/

! edgetpu_compiler model_quantized.tflite

/content/training/fine_tuned_model
Edge TPU Compiler version 2.1.302470888

Model compiled successfully in 626 ms.

Input model: model_quantized.tflite
Input size: 4.54MiB
Output model: model_quantized_edgetpu.tflite
Output size: 5.20MiB
On-chip memory used for caching model parameters: 5.05MiB
On-chip memory remaining for caching model parameters: 2.58MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 99
Operation log: model_quantized_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 98
Number of operations that will run on CPU: 1
See the operation log file for individual operation details.


When the compilation is done, all you have to do is to use the Edge TPU-optimized file with your Coral device!

## Download files

In [0]:
# Compress into an archive
%cd /content/training
!zip -r fine_tuned_model.zip fine_tuned_model

/content/training
  adding: fine_tuned_model/ (stored 0%)
  adding: fine_tuned_model/tflite_graph.pb (deflated 9%)
  adding: fine_tuned_model/model.ckpt.meta (deflated 94%)
  adding: fine_tuned_model/pipeline.config (deflated 70%)
  adding: fine_tuned_model/model_float.tflite (deflated 72%)
  adding: fine_tuned_model/frozen_inference_graph_image_tensor.pb (deflated 11%)
  adding: fine_tuned_model/checkpoint (deflated 42%)
  adding: fine_tuned_model/tflite_graph.pbtxt (deflated 56%)
  adding: fine_tuned_model/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: fine_tuned_model/saved_model/ (stored 0%)
  adding: fine_tuned_model/saved_model/variables/ (stored 0%)
  adding: fine_tuned_model/saved_model/saved_model.pb (deflated 11%)
  adding: fine_tuned_model/frozen_inference_graph_encoded_image_string_tensor.pb (deflated 11%)
  adding: fine_tuned_model/model.ckpt.index (deflated 67%)
  adding: fine_tuned_model/model_quantized.tflite (deflated 28%)
  adding: fine_tuned_model/label_map.p

In [0]:
# Download the archive
# If it does not work: Just right-click the file in the left-side file browser and download it from the context menu
%cd /content/training
from google.colab import files
files.download("fine_tuned_model.zip")

/content/training


KeyboardInterrupt: ignored

## Use it with Coral



The fastest way to try it, is to use the model with code from the Coral examples, such as the [official example of object detection](https://coral.ai/examples/detect-image/).

Here's an example (use your own paths, obviously):

```
python3 detect_image.py \
  --model models/YOUR_MODEL_quantized_edgetpu.tflite \
  --label models/YOUR_MODEL_labels.txt \
  --input images/TEST_IMAGE.jpg \
  --output ${HOME}/object_detection_results.jpg
```